In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import zipfile
import shutil
import ast

Este notebook esta preparado para trabajar desde google drive, por favor, asegurese de que el notebook se encuentre en el directorio donde haya guardado las carpetas con las imagenes o ajuste sus rutas a la hora de leer los archivos

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!ls "/content/drive/My Drive"

In [14]:
os.getcwd()
os.chdir('D:\Datos Usuario\Desktop\data\Año 2.2\TFM\Imagenes')

# ANALISIS EXPLORATORIO

In [15]:
#Lectura del archivo que contiene las etiquetas de todas las imagenes
df = pd.read_csv("PADCHEST_chest_x_ray_images_labels_160K_01.02.19 (1).csv")

n_rows = print(f'El número total de imagenes es:',len(df))
n_cols = print(f'Informacion almacenada de cada imagen:',df.columns)

El número total de imagenes es: 160861
Informacion almacenada de cada imagen: Index(['Unnamed: 0', 'ImageID', 'ImageDir', 'StudyDate_DICOM', 'StudyID',
       'PatientID', 'PatientBirth', 'PatientSex_DICOM', 'ViewPosition_DICOM',
       'Projection', 'MethodProjection', 'Pediatric', 'Modality_DICOM',
       'Manufacturer_DICOM', 'PhotometricInterpretation_DICOM',
       'PixelRepresentation_DICOM', 'PixelAspectRatio_DICOM',
       'SpatialResolution_DICOM', 'BitsStored_DICOM', 'WindowCenter_DICOM',
       'WindowWidth_DICOM', 'Rows_DICOM', 'Columns_DICOM',
       'XRayTubeCurrent_DICOM', 'Exposure_DICOM', 'ExposureInuAs_DICOM',
       'ExposureTime', 'RelativeXRayExposure_DICOM', 'ReportID', 'Report',
       'MethodLabel', 'Labels', 'Localizations',
       'LabelsLocalizationsBySentence', 'labelCUIS', 'LocalizationsCUIS'],
      dtype='object')


C:\Users\Usuario\AppData\Local\Temp\ipykernel_5224\3465690643.py:2: DtypeWarning: Columns (19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("PADCHEST_chest_x_ray_images_labels_160K_01.02.19 (1).csv")


In [16]:
#Recuento de imagenes sin patologia
df['Labels'] = df['Labels'].fillna("[]")
df['Labels'] = df['Labels'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df['Normal'] = df['Labels'].apply(lambda labels: 'normal' in labels)
n_normal = df['Normal'].sum()

#Recuento de imagenes con metastasis pulmonar
df['Labels'] = df['Labels'].fillna("[]")
df['Labels'] = df['Labels'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df['Has_Lung_Metastasis'] = df['Labels'].apply(lambda labels: 'lung metastasis' in labels)
n_lung_metastasis = df['Has_Lung_Metastasis'].sum()

print(f"Número de entradas con etiquetas 'normal': {n_normal}")
print(f"Número de entradas con etiquetas 'lung metastasis': {n_lung_metastasis}")

Número de entradas con etiquetas 'normal': 50340
Número de entradas con etiquetas 'lung metastasis': 326


Observamos que en total tenemos 50340 imagenes sin patologia y solo 326 con patología, esto muestra que hay un claro desbalanceo entre las clases de las imagenes que habrá que mitigar.

A continuación analizamos dentro de las imagenes con metastasis los distintos tipos de proyeccion de imagen.

In [17]:
# Imagenes con metastasis
df_lung_metastasis = df[df['Has_Lung_Metastasis']]

# Recuento de valores en la columna 'Projection'
projection_counts = df_lung_metastasis['Projection'].value_counts()

print("Recuento de valores para la columna 'Projection' en imágenes con 'lung metastasis':")
print(projection_counts)

Recuento de valores para la columna 'Projection' en imágenes con 'lung metastasis':
PA               165
L                118
AP_horizontal     31
AP                 9
COSTAL             3
Name: Projection, dtype: int64
